# Some imports

In [1]:
!pip install tensordict
!pip install torchviz
!pip install bdpy
from torchvision.models import convnext_tiny,ConvNeXt_Tiny_Weights
from torchvision.io import read_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.5/986.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=c5e32fade7ad18009273d2cb5a5a81863d1185f6973feecebe477d1d87764375
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00


In [2]:
import csv
import os
import pickle
import re
import imghdr
from time import time
import torch
import numpy as np
import pandas as pd
import PIL.Image
import bdpy

In [3]:
# Testing convnext for class prediction on imagenet
convnext_weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1
model=convnext_tiny(weights=convnext_weights)
#print(model)
model.eval()

preprocess=convnext_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)

prediction=model(batch).squeeze(0).softmax(0)
class_id= prediction.argmax().item()
score=prediction[class_id].item()
category_name=convnext_weights.meta["categories"][class_id]
print(f"{category_name} : {score*100} % ")

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 308MB/s] 
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


African elephant : 35.99933981895447 % 


In [4]:
import torch
convnext = convnext_tiny(weights=convnext_weights)

convnext_no_fc= torch.nn.Sequential(*list(convnext.children())[:-1]) # strips off last linear layer
#print(convnext_no_fc)
preprocess=convnext_weights.transforms()
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)

prediction=convnext_no_fc(batch)
# Convert pytorch model to flow of data diagram
from torchviz import make_dot, make_dot_from_trace
model_dia=make_dot(convnext(batch),params=dict(convnext_no_fc.named_parameters()), show_attrs=True, show_saved=True)
model_dia.format="png"
model_dia.render("convnext_model_dia")
print("#####################SHOW THE MODEL NODES##############")
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
train_nodes, eval_nodes = get_graph_node_names(convnext)
for i,j in enumerate(train_nodes):
    print(f"Train node {i} : {j}")
    
print("##############THE NODES TO BE SELECTED FROM THE CONVNEXT MODEL#############################")
list_of_layers_index=[9,48,87,120,153,175,192,225]
layers_to_be_printed=dict()
count=0
for i,j in enumerate(train_nodes):
    if i in list_of_layers_index:
        print(f"Train node {i} : {j}")
        count+=1
        layers_to_be_printed[j]=f"layer{count}"
convnext_return_nodes=layers_to_be_printed
convnext_feature_extractor=create_feature_extractor(convnext,return_nodes=convnext_return_nodes)
img=read_image("/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/test/ILSVRC2012_test_00000046.JPEG")
batch=preprocess(img).unsqueeze(0)
feature_list=convnext_feature_extractor(batch)
print("##########################PRINTING THE SHAPES OF THE FEATURES TO BE EXRACTED###############")
for x in feature_list:
    print(torch.flatten(feature_list[x]).shape)

#####################SHOW THE MODEL NODES##############
Train node 0 : x
Train node 1 : features.0
Train node 2 : features.1.0.features_1_0_layer_scale
Train node 3 : features.1.0.block.0
Train node 4 : features.1.0.block.1
Train node 5 : features.1.0.block.2
Train node 6 : features.1.0.block.3
Train node 7 : features.1.0.block.4
Train node 8 : features.1.0.block.5
Train node 9 : features.1.0.block.6
Train node 10 : features.1.0.mul
Train node 11 : features.1.0.stochastic_depth
Train node 12 : features.1.0.add
Train node 13 : features.1.1.features_1_1_layer_scale
Train node 14 : features.1.1.block.0
Train node 15 : features.1.1.block.1
Train node 16 : features.1.1.block.2
Train node 17 : features.1.1.block.3
Train node 18 : features.1.1.block.4
Train node 19 : features.1.1.block.5
Train node 20 : features.1.1.block.6
Train node 21 : features.1.1.mul
Train node 22 : features.1.1.stochastic_depth
Train node 23 : features.1.1.add
Train node 24 : features.1.2.features_1_2_layer_scale
Train

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [5]:
print(convnext_return_nodes)

{'features.1.0.block.6': 'layer1', 'features.3.0.block.6': 'layer2', 'features.5.0.block.6': 'layer3', 'features.5.3.block.6': 'layer4', 'features.5.6.block.6': 'layer5', 'features.5.8.block.6': 'layer6', 'features.7.0.block.6': 'layer7', 'classifier.2': 'layer8'}


In [6]:
'''
makedata_features
Extarce CNN features from images and save them in a BData file (hdf5).
This file is a part of GenericDecoding_demo.
'''
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential
from sklearn.decomposition import PCA
# Functions ##################################################################
pca_func_per_layer_list=[PCA(n_components=1000) for x in range(0,8)]
def get_feature(model, images, layers,image_dir,feature_num=0,batch_size=1,is_test=True):
    '''
    Returns CNN features
    '''

    # Convert 'images' to a list
    if not isinstance(images, list):
        images = [images]

    num_images = len(images)
    

    image_index = [i for i in range(0,num_images)]

#     mean_image = self.net.transformer.mean['data']

    
    layerwise_flattened_feature_list=[[] for x in range(0,8)]
    
    for i, imgind in enumerate(image_index):
    
        img_path = os.path.join(image_dir, images[imgind])
        transforms=convnext_weights.transforms()
        img = torch.unsqueeze(transforms(read_image(images[imgind],mode=torchvision.io.ImageReadMode.RGB)).to("cpu"),0)
        
        feature_list=convnext_feature_extractor(img)

        for layer_index,x in enumerate(feature_list):
            #print(feature_list[x].shape)
            y=torch.flatten(feature_list[x]).detach().cpu().numpy()#flatten in code not working
            #print(np.expand_dims(y,0).shape)
            
            layerwise_flattened_feature_list[layer_index].append(y)
            #print(flattened_feature_dict[x].shape)
        
    
#     print("Total number of image features :",len(feature_all))
#     print("Number of layer per image :",len(feature_all[0]))
#     print("Shape of each layer:",feature_all[0][0].shape)
    layerwise_flattened_feature_list= [np.array(layer_features) for layer_features 
                                                                                 in layerwise_flattened_feature_list]
    device = torch.device("cuda")  
    reduced_data_layer_list=[]
    for layer in range(0,8):

        x=layerwise_flattened_feature_list[layer]
        print("over all samples each layer shape:",x.shape) 
        if is_test==False:
            pca_layer_data = pca_func_per_layer_list[layer].fit_transform(x)
        elif is_test==True:
            pca_layer_data = pca_func_per_layer_list[layer].transform(x)
        reduced_data_layer_list.append(pca_layer_data)

    feature_all = np.array(reduced_data_layer_list)
    print("Reduced data shape:",feature_all.shape)
    feature_all = np.expand_dims(np.transpose(feature_all,(1,0,2)),2)
    print("Reduced and transformed data shape:",feature_all.shape)
    return feature_all

def get_image_features(model, imagedir, layers=[], n_features=1000, save=False, savefile=None,is_test_data=True):
    '''Calculate image features in `imagedir` and save them in `saveile`'''

    if os.path.exists(savefile):
        print('already exists. Loading the results.',savefile)
        with open(savefile, 'rb') as f:
            feat = pickle.load(f)
        return feat

    ## Get image files
    imagefiles = []
    for root, dirs, files in os.walk(imagedir):
        imagefiles = [os.path.join(root, f)
                      for f in files
                      if imghdr.what(os.path.join(root, f))]

    #print ('Total Number of Images sent to get_image_features:',len(imagefiles))

    #print( 'Getting features')
  
    features = get_feature(model,imagefiles, layers,imagedir,feature_num=n_features,is_test=is_test_data)
    
    #print("###########################")

    # Convert to pandas dataframe
    feat = pd.DataFrame([[lay for lay in img] for img in features],
                        index=[os.path.split(f)[1] for f in imagefiles],
                        columns=layers)
    
    # Save data in a pickle file
    if save:
        with open(savefile, 'wb') as f:
            pickle.dump(feat, f)
        print ('Saved:',savefile)

    return feat

def get_category_averaged_features(net, imagedir, layers=[], n_features=1000, save=False, savefile=None,is_test_data_flag=True):
    '''Calculate category averaged features'''

    feat_dir, feat_file = os.path.split(savefile)

    catlist = os.listdir(imagedir)

    dat = pd.DataFrame(index=catlist, columns=layers)

    for cat in catlist:
        catdir = os.path.join(imagedir, cat)
        catfile = os.path.join(feat_dir, feat_file + '-' + cat + '.pkl')

        if not os.path.isdir(catdir):
            continue

        print ('Category:',cat )

        feat = get_image_features(net, catdir,
                                  layers=layers, n_features=n_features,
                                  save=True, savefile=catfile,is_test_data=is_test_data_flag)

        # Calc mean
        print('Calculating averaged features')
        layers = feat.columns
        for lay in layers:
            feat_mean = np.mean(np.vstack([f for f in feat[lay]]), axis=0)
            dat[lay][cat] = feat_mean

    if save:
        with open(savefile, 'wb') as f:
            pickle.dump(dat, f)
        print ('Saved:', savefile)
    print(f"\n \n Avgeraged category feature shape :",dat.shape)
    return dat

def get_category_id(image_filename):
    return int(image_filename.split('_')[0][1:])


def get_image_id(image_filename):
    cat_id = get_category_id(image_filename)
    img_id = int(image_filename.split('_')[1][:-5])
    return float('%d.%06d' % (cat_id, img_id))








In [7]:

def main():
    # Settings ---------------------------------------------------------------

    # Feture selection settings
    num_features = 1000
    
    # CNN model settings
    cnn_layers = ['layer1', 'layer2', 'layer3', 'layer4', 'layer5', 'layer6', 'layer7', 'layer8']

    #mean_image_file = './data/images/ilsvrc_2012_mean.npy' # ImageNet Large Scale Visual Recognition Challenge 2012

    # Results file
    data_dir = '/kaggle/working/'
    featuredir = os.path.join(data_dir, 'ImageFeatures/')
    outputfile = os.path.join(data_dir, 'ImageFeatures_pytorch.h5')

    # Misc settings
    rand_seed = 2501

    # Preparation ------------------------------------------------------------

    # Caffe GPU setting
    #caffe.set_mode_gpu()

    # Result directory
    if not os.path.exists(featuredir):
        os.makedirs(featuredir)

    # Load CNN model ---------------------------------------------------------
    model = create_feature_extractor(convnext,return_nodes=convnext_return_nodes)

    # Get image features for traning images ----------------------------------
    print("Generating training img features :")
    features_train = get_image_features(model, '/kaggle/input/seen-vs-imagined-horikawa-et-al-imagenet-subset/images/images/training',
                                        layers=cnn_layers, n_features=num_features,
                                        save=True, savefile=os.path.join(featuredir, 'features_training.pkl'),is_test_data=False)
    
    print("Generating testing img features ")
    # Get image features for test images -------------------------------------
    features_test = get_image_features(model, '/kaggle/input/seen-vs-imagined-horikawa-et-al-imagenet-subset/images/images/test',
                                       layers=cnn_layers, n_features=num_features,
                                       save=True, savefile=os.path.join(featuredir, 'features_test.pkl'),is_test_data=True)
    
    # Category averaged features ---------------------------------------------
    print("Generating cat avg img features ")
    features_cat_test = get_category_averaged_features(model, '/kaggle/input/seen-vs-imagined-horikawa-et-al-imagenet-subset/imagenet_cat_avg_imagery_subset/kaggle/working/imagenet_cat_avg_imagery_subset',
                                                       layers=cnn_layers, n_features=num_features,
                                                       save=True, savefile=os.path.join(featuredir, 'feature_category_ave_test.pkl'),is_test_data_flag=True)

    
    # Create features data in BData ------------------------------------------
    features = bdpy.BData()
    
    
    #features_list = [features_train, features_test, features_cat_test, features_cat_cand]
    features_list = [features_train, features_test,features_cat_test]
    
    #add code for genrating Iain TDA features from convnext
    
    
    featuretype_arrays = []
    categoryid_arrays = []
    imageid_arrays = []
    features_dict = {lay : [] for lay in cnn_layers}

    for i, feat in enumerate(features_list):
        n_sample = len(feat.index)
        
        feature_type = i + 1

        image_id = feat.index

        for img in image_id:
            cat_id = get_category_id(img)
            if i < 2:
                img_id = get_image_id(img)
            else:
                img_id = np.nan

            featuretype_arrays.append(feature_type)
            categoryid_arrays.append(cat_id)
            imageid_arrays.append(img_id)

            for lay in cnn_layers:
                print(feat[lay][img].dtype)
                print(feat[lay][img].shape)
                features_dict[lay].append(feat[lay][img])
                
    for q in range(len(featuretype_arrays)):
        print("Feature type :",featuretype_arrays[q])
        print("CatID :",categoryid_arrays[q])
        print("ImageID :",imageid_arrays[q])
        
    # Add data
    features.add(np.vstack(featuretype_arrays), 'FeatureType')
    features.add(np.vstack(categoryid_arrays), 'CatID')
    features.add(np.vstack(imageid_arrays), 'ImageID')
    for ft in features_dict:
        print("\n###################per layer cnn feature####################\n")
        #print(f"for feature {ft} shape :",features_dict[ft].shape)
        
        #print(np.vstack(features_dict[ft]).shape)
        features.add(np.vstack(features_dict[ft]), ft)
    
    # Save merged features ---------------------------------------------------
    print(features.show_metadata())
    features.save(outputfile)
    




if __name__ == '__main__':
    main()

Generating training img features :


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


over all samples each layer shape: (1200, 301056)
over all samples each layer shape: (1200, 150528)
over all samples each layer shape: (1200, 75264)
over all samples each layer shape: (1200, 75264)
over all samples each layer shape: (1200, 75264)
over all samples each layer shape: (1200, 75264)
over all samples each layer shape: (1200, 37632)
over all samples each layer shape: (1200, 1000)
Reduced data shape: (8, 1200, 1000)
Reduced and transformed data shape: (1200, 8, 1, 1000)
Saved: /kaggle/working/ImageFeatures/features_training.pkl
Generating testing img features 


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


over all samples each layer shape: (50, 301056)
over all samples each layer shape: (50, 150528)
over all samples each layer shape: (50, 75264)
over all samples each layer shape: (50, 75264)
over all samples each layer shape: (50, 75264)
over all samples each layer shape: (50, 75264)
over all samples each layer shape: (50, 37632)
over all samples each layer shape: (50, 1000)
Reduced data shape: (8, 50, 1000)
Reduced and transformed data shape: (50, 8, 1, 1000)
Saved: /kaggle/working/ImageFeatures/features_test.pkl
Generating cat avg img features 
Category: n03452741


...

over all samples each layer shape: (2, 301056)
over all samples each layer shape: (2, 150528)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 37632)
over all samples each layer shape: (2, 1000)
Reduced data shape: (8, 2, 1000)
Reduced and transformed data shape: (2, 8, 1, 1000)
Saved: /kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n01976957.pkl
Calculating averaged features
Category: n02950256


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


over all samples each layer shape: (2, 301056)
over all samples each layer shape: (2, 150528)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 75264)
over all samples each layer shape: (2, 37632)
over all samples each layer shape: (2, 1000)
Reduced data shape: (8, 2, 1000)
Reduced and transformed data shape: (2, 8, 1, 1000)
Saved: /kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n02950256.pkl
Calculating averaged features
Category: n02690373


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


over all samples each layer shape: (1300, 301056)
over all samples each layer shape: (1300, 150528)
over all samples each layer shape: (1300, 75264)
over all samples each layer shape: (1300, 75264)
over all samples each layer shape: (1300, 75264)
over all samples each layer shape: (1300, 75264)
over all samples each layer shape: (1300, 37632)
over all samples each layer shape: (1300, 1000)
Reduced data shape: (8, 1300, 1000)
Reduced and transformed data shape: (1300, 8, 1, 1000)
Saved: /kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n02690373.pkl
Calculating averaged features
Saved: /kaggle/working/ImageFeatures/feature_category_ave_test.pkl

 
 Avgeraged category feature shape : (50, 8)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
float32
(1, 1000)
flo

In [8]:
!zip -r "kamitani_style_img_features.zip"  "/kaggle/working/ImageFeatures_pytorch.h5" "/kaggle/working/ImageFeatures/"

  adding: kaggle/working/ImageFeatures_pytorch.h5 (deflated 46%)
  adding: kaggle/working/ImageFeatures/ (stored 0%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n02128385.pkl (deflated 9%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n02882301.pkl (deflated 53%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n04533802.pkl (deflated 53%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n04572121.pkl (deflated 53%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n03954393.pkl (deflated 20%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n03495258.pkl (deflated 9%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n03710193.pkl (deflated 9%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n03345837.pkl (deflated 20%)
  adding: kaggle/working/ImageFeatures/feature_category_ave_test.pkl-n04297750.pkl (deflated 31%)
  ad

In [9]:
!rm -rf /kaggle/working/ImageFeatures
!rm /kaggle/working/ImageFeatures_pytorch.h5  

In [10]:
# have to figure out at what part of the code all the features for all the latyers are being available as a numpy array the shape is (1400,8,1,1000)